In [1]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [2]:
from classes.grid import Grid
import re

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

# Part 1

In [3]:
class RobotGrid(Grid):
    def __init__(self, grid):
        super().__init__(grid)
        self.robots = set()
    
    def update_robot_positions(self):
        '''
        Uses the positions of all robots in the grid to show how many are currently in each tile
        '''
        # Reset grid before we place the new iteration of robots
        self.grid = [['.']*self.num_cols for _ in range(self.num_rows)]
        # Place each robot into the grid and add 1 to the tile that it is placed in
        for robot in self.robots:
            cur_tile = self.grid[robot.row][robot.col]
            if cur_tile == '.':
                self.grid[robot.row][robot.col] = '1'
            else:
                self.grid[robot.row][robot.col] = str(int(cur_tile) + 1)
    
    def move_all_robots(self, iterations=1):
        '''
        Moves all robots for the given number of iterations and updates their positions in the grid
        '''
        for _ in range(iterations):
            for robot in self.robots:
                robot.move()
            self.update_robot_positions()
    
    def safety_factor(self):
        '''
        Returns the safety factor of the robot grid (quadrant robot count product)
        '''
        # Split into quadrants
        mid_row = self.num_rows // 2
        mid_col = self.num_cols // 2
        
        quad_1 = quad_2 = quad_3 = quad_4 = 0

        for row in range(self.num_rows):
            for col in range(self.num_cols):
                cur_tile = self.grid[row][col]
                if cur_tile != '.':
                    if row < mid_row and col < mid_col:
                        quad_1 += int(cur_tile)
                    elif row > mid_row and col < mid_col:
                        quad_2 += int(cur_tile)
                    elif row < mid_row and col > mid_col:
                        quad_3 += int(cur_tile)
                    elif row > mid_row and col > mid_col:
                        quad_4 += int(cur_tile)
        
        return quad_1*quad_2*quad_3*quad_4
    

class Robot():
    def __init__(self, col, row, h_vel, v_vel, grid: RobotGrid):
        self.row = int(row)
        self.col = int(col)
        self.h_vel = int(h_vel)
        self.v_vel = int(v_vel)
        self.grid = grid
    
    def move(self):
        '''
        Moves this robot to its next position (where it is after 1 second)
        '''
        if self.h_vel >= 0:
            h_movement = self.grid.get_relative(self.row,self.col,'right',step=self.h_vel,wraparound=True)
        else:
            h_movement = self.grid.get_relative(self.row,self.col,'left',step=self.h_vel*(-1),wraparound=True)

        if self.v_vel >= 0:
            movement = self.grid.get_relative(h_movement[0],h_movement[1],'down',step=self.v_vel,wraparound=True)
        else:
            movement = self.grid.get_relative(h_movement[0],h_movement[1],'up',step=self.v_vel*(-1),wraparound=True)

        self.row = movement[0]
        self.col = movement[1]

In [4]:
robots_info = input.split('\n')
robots_info

# Identify grid dimensions
if input == example:
    grid_rows = 7
    grid_cols = 11
elif input == puzzle:
    grid_rows = 103
    grid_cols = 101

# Set up a grid to operate on
ebhq_grid = RobotGrid(['.'*grid_cols for _ in range(grid_rows)])

# Initialise Robots
for robot in robots_info:
    # Extract info
    robot_x = re.findall(r'p=(\d*)', robot)[0]
    robot_y = re.findall(r'p=\d*,(\d*)', robot)[0]
    robot_h_vel = re.findall(r'v=(-*\d*)', robot)[0]
    robot_v_vel = re.findall(r'v=-*\d*,(-*\d*)', robot)[0]

    cur_robot = Robot(robot_x,robot_y,robot_h_vel, robot_v_vel, ebhq_grid)
    ebhq_grid.robots.add(cur_robot)

# Iterate 100 seconds
ebhq_grid.move_all_robots(iterations=100)

#ebhq_grid.print_grid()
ebhq_grid.safety_factor()


225521010

# Part 2

In [5]:


robots_info = input.split('\n')
robots_info

# Identify grid dimensions
if input == example:
    grid_rows = 7
    grid_cols = 11
elif input == puzzle:
    grid_rows = 103
    grid_cols = 101

# Set up a grid to operate on
ebhq_grid = RobotGrid(['.'*grid_cols for _ in range(grid_rows)])

# Initialise Robots
for robot in robots_info:
    # Extract info
    robot_x = re.findall(r'p=(\d*)', robot)[0]
    robot_y = re.findall(r'p=\d*,(\d*)', robot)[0]
    robot_h_vel = re.findall(r'v=(-*\d*)', robot)[0]
    robot_v_vel = re.findall(r'v=-*\d*,(-*\d*)', robot)[0]

    cur_robot = Robot(robot_x,robot_y,robot_h_vel, robot_v_vel, ebhq_grid)
    ebhq_grid.robots.add(cur_robot)

# Part 2...

# The output relies on visual confirmation that we have the right image
# Taking the hint from part 1, use the safety factor; if an image is formed of robots
# spread out such that certain quadrants have very few, then this will in turn decrease the safety factor.
# Hence, I have taken the safety factor from part 1 as a reference and have used it as a limit to filter out
# certain grid formations. Initially reducing it by 2 orders of magnitude and slowly increasing this until
# grids started printing from the code below, you can see the Christmas tree appear after 7774 seconds 

for second in range(10000):
    ebhq_grid.move_all_robots()
    if ebhq_grid.safety_factor() < 99999999:
        #ebhq_grid.print_grid()
        print(f'above is after {second+1} seconds')


above is after 3963 seconds
above is after 7774 seconds
above is after 8598 seconds
above is after 9895 seconds
